# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.847750717977                   -0.70    4.8   30.6ms
  2   -7.852401839074       -2.33       -1.53    1.0   17.3ms
  3   -7.852607366913       -3.69       -2.54    1.0   17.0ms
  4   -7.852645342785       -4.42       -2.78    2.2   67.3ms
  5   -7.852646071392       -6.14       -2.87    1.0   17.3ms
  6   -7.852646669005       -6.22       -3.98    1.0   17.1ms
  7   -7.852646686252       -7.76       -4.53    1.8   20.4ms
  8   -7.852646686705       -9.34       -5.35    1.2   18.3ms
  9   -7.852646686719      -10.84       -5.27    1.8   19.8ms
 10   -7.852646686729      -11.01       -5.87    1.0   17.9ms
 11   -7.852646686730      -12.08       -6.86    1.0   17.4ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.847785866858                   -0.70           4.8    552ms
  2   -7.852561326627       -2.32       -1.62   0.80    2.0    498ms
  3   -7.852641492499       -4.10       -2.69   0.80    1.0    301ms
  4   -7.852646529901       -5.30       -3.38   0.80    1.8   19.3ms
  5   -7.852646681225       -6.82       -4.37   0.80    1.8   18.5ms
  6   -7.852646686620       -8.27       -4.82   0.80    2.2   21.2ms
  7   -7.852646686726       -9.97       -5.51   0.80    1.0   15.9ms
  8   -7.852646686730      -11.48       -6.70   0.80    1.5   17.8ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.427035e+01     3.444237e+00
 * time: 0.04597187042236328
     1     1.070274e+00     1.768442e+00
 * time: 0.27100205421447754
     2    -1.591883e+00     2.139380e+00
 * time: 0.2886168956756592
     3    -3.800034e+00     1.782293e+00
 * time: 0.3139040470123291
     4    -5.025576e+00     1.987891e+00
 * time: 0.3391458988189697
     5    -6.761978e+00     1.198385e+00
 * time: 0.36451292037963867
     6    -7.328938e+00     4.610408e-01
 * time: 0.38998985290527344
     7    -7.572582e+00     3.017163e-01
 * time: 0.4076390266418457
     8    -7.669602e+00     1.218459e-01
 * time: 0.4253678321838379
     9    -7.731643e+00     1.218257e-01
 * time: 0.44299793243408203
    10    -7.771973e+00     1.245007e-01
 * time: 0.46068787574768066
    11    -7.808097e+00     8.620072e-02
 * time: 0.4784209728240967
    12    -7.834978e+00     4.329769e-02
 * time: 0.4963810443878174
    13    -7.848132e+00     2.315259e-02
 * time: 0.5141

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.847766927299                   -0.70    4.5   30.3ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645831302                   -1.63    3.65s
  2   -7.852646686730       -6.07       -3.69    1.68s
  3   -7.852646686730      -13.19       -7.19    124ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.3334047560673926e-7
|ρ_newton - ρ_scfv| = 4.103275347485547e-7
|ρ_newton - ρ_dm|   = 7.369658346262652e-10
